In [ ]:
# let user input dataset name

import pandas as pd
import os
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
dataset_name = input("Enter dataset name: ")

# Search file with levinstein distance

dataset_files = []
for root, dirs, files in os.walk("data"):
    for file in files:
        if file.endswith(".csv"):
            dataset_files.append(file)


def calculate_levenshtein(str1, str2):
    replacementCost = [[]]

    for i in range(len(str1) + 1):
        replacementCost.append([])
        for j in range(len(str2) + 1):
            if i == 0:
                replacementCost[i].append(j)
            elif j == 0:
                replacementCost[i].append(i)
            elif str1[i - 1] == str2[j - 1]:
                replacementCost[i].append(replacementCost[i - 1][j - 1])
            else:
                replacementCost[i].append(1 + min(replacementCost[i - 1][j], replacementCost[i][j - 1],
                                                  replacementCost[i - 1][j - 1]))

    return replacementCost[len(str1)][len(str2)]


# Find the closest file
scores = []

for file in dataset_files:
    print(file)
    scores.append(calculate_levenshtein(dataset_name, file.split("_data")[0]))

closest_file_name = dataset_files[scores.index(min(scores))]
print("Closest file: " + closest_file_name)

# Load the dataset
df = pd.read_csv("data/" + closest_file_name)

df = df.drop(columns=["imdb_id", "id"])

# print(df)
df["score_avg"] = (df["imdb_score"] + df["tmdb_score"]) / 2

In [ ]:
df["genres_encoded"] = df["genres"].astype("category").cat.codes
df[["genres", "genres_encoded"]].astype("category")

df["type_encoded"] = df["type"].astype("category").cat.codes
df[["type", "type_encoded"]].astype("category")

df["age_certification_encoded"] = df["age_certification"].astype("category").cat.codes
df[["age_certification", "age_certification_encoded"]].astype("category")

df["production_countries_encoded"] = df["production_countries"].astype("category").cat.codes
df[["production_countries", "production_countries_encoded"]].astype("category")

In [ ]:
# set seasons to 0 if it is NaN
df["seasons"] = df["seasons"].fillna(0)

# remove nan values
df = df.dropna()

# df["imdb_score"] = int(df["imdb_score"] * 10)
df["imdb_score"] = df["imdb_score"] * 10
print(df["imdb_score"])
df["imdb_score"] = int(df["imdb_score"])
df["tmdb_score"] = int(df["tmdb_score"] * 10)
df["tmdb_popularity"] = int(df["tmdb_popularity"] * 100)
df["score_avg"] = int(df["score_avg"] * 10)


# Split in train and test set
X = df.drop(columns=['score_avg', 'title', 'genres', 'type', 'age_certification', 'production_countries', 'description'])
y = df['score_avg']
print(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

score = accuracy_score(y_test, predictions)
score
